In [101]:
import numpy as np
#import tensorflow as tf
#from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

from sklearn.model_selection import cross_val_score

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10




In [147]:
data=pd.read_csv('dataCompletTo20mai.csv', sep=",", encoding = "ISO-8859-1")
data=data.set_index('time')

#Linky=pd.read_csv('Enedis_Conso_Heure_20210325-20210427_09311432680380.csv', sep=";", encoding = "ISO-8859-1")

In [148]:
data

,Temperature,Humidity,CO2,Noise,Pressure,activity
time,,,,,,
1617235589,19.9,56,647.0,32.0,1017.5,Present
1617235891,19.9,56,602.0,32.0,1017.5,Present
1617236194,19.9,56,609.0,32.0,1017.6,Present
1617236497,19.9,56,613.0,32.0,1017.5,Present
1617236799,19.8,56,624.0,32.0,1017.5,Present
1617237102,19.8,55,602.0,32.0,1017.5,Present
1617237405,19.8,55,589.0,32.0,1017.5,Present
1617237707,19.8,55,580.0,32.0,1017.5,Present
1617238010,19.8,55,613.0,32.0,1017.5,Present


In [149]:
def lagged(df,lag=3):
    df=df.set_index('time')    
    Df=[]
    a=[]
    for i in range(lag):
        a.append(i)
        df1=df.shift(i+1).copy()
        df1=df1.add_suffix('_t-'+str(i+1))
        Df.append(df1)
    for base in Df:
        df=pd.concat([df,base], axis=1)
    
    df=df.drop(df.index[a])
    return df, a 

In [151]:
# Création de jour de semaine (d.isoweekday()) et l'heure de la journée (d.hour+d.minute/60) à partire de timestamp
import datetime
Jsemaine =[]
heure=[]
for t in data.index:
    d=datetime.datetime.fromtimestamp(t)
    Jsemaine.append(d.isoweekday())
    heure.append(d.hour+d.minute/60)

In [174]:
# afficher l’importance des variables de notre modèle de forêt aléatoire
pd.DataFrame(rf_model.feature_importances_,
              index = X_train.columns, 
              columns = ["importance"]).sort_values(
     "importance", 
     ascending = False)

,importance
CO2_t-3,0.149056
CO2_t-1,0.130488
CO2_t-2,0.122158
CO2,0.112003
heureday,0.089778
Temperature_t-2,0.066762
Temperature,0.057978
Temperature_t-3,0.046870
Temperature_t-1,0.032599
Humidity_t-2,0.018304


In [154]:
# afficher l’importance des variables de notre modèle de forêt aléatoire
pd.DataFrame(rf_model.feature_importances_,
              index = X_train.columns, 
              columns = ["importance"]).sort_values(
     "importance", 
     ascending = False)

,importance
CO2_t-3,0.375392
Temperature_t-2,0.213277
heureday,0.094873
CO2_t-1,0.033400
CO2_t-2,0.032711
Temperature_t-1,0.027625
Humidity_t-2,0.022864
Temperature_t-3,0.022078
CO2,0.020873
Pressure_t-1,0.020322


# Matrice de confusion

In [155]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, rf_model.predict(X_test)),
             index = ["Present_data", "Abscent_data"],
             columns = ["Present_predit", "Abscent_predit"])

,Present_predit,Abscent_predit
Present_data,264,86
Abscent_data,628,1902


In [156]:
from sklearn import metrics

y_pred = rf_model.predict(X_test)
y_true = y_test
metrics.precision_recall_fscore_support(y_true, y_pred, beta=1)

(array([0.29596413, 0.95674044]),
 array([0.75428571, 0.75177866]),
 array([0.42512077, 0.84196547]),
 array([ 350, 2530], dtype=int64))

## Random Forest for all dataImputed 

In [176]:
RANDOM_SEED = 20
import os
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

test_size=0.20
lagg=10
#nbObservation=14400
nbObservation=14400
scaler = StandardScaler()
datascaler=scaler.fit(data.values[:,1:-1])
n=10
# list = os.listdir('dataImputed')
# for fichier in list:
directory = 'dataImputed'
lagg=3
# iterate over files in
# that directory
print("dataImputed",";","accuracy.mean()",";","accuracy.std()",";","fscore.mean()",";","fscore.std()")
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        df = pd.read_csv(f, sep=",", encoding = "ISO-8859-1")        
        #df= pd.DataFrame(datascaler.inverse_transform(df.values[:,1:]),columns=df.columns[1:])      
        
        df, lignesAsupprimer=lagged(df,lagg)        
        #print(df.columns)
        data1=data.copy()       
        df["activity"] = data1.activity.drop(data1.index[lignesAsupprimer]) # supprimer les premières lignes
        df['activity'].replace(['Absent','Present'],[0,1], inplace=True)
        df["JourSemaine"]=Jsemaine[len(lignesAsupprimer):]
        df["heureday"]=heure[len(lignesAsupprimer):]
        #print(df.columns)
        db2=df.copy()      
        X=db2.copy()       
        X=X.drop('activity', axis=1)
        y=db2['activity']
        #print(y.head(2))
        X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=test_size,shuffle=False)          
        accuracy=np.zeros(n)
        fscore=np.zeros(n)
        accB=np.zeros(n)
        for i in range(n):
            random.seed(i)
            rf_model=RandomForestClassifier(max_features='auto', n_estimators= 10, max_depth=None, criterion='entropy')
            rf_model.fit(X_train,y_train)           
            y_true = y_test
            y_pred = rf_model.predict(X_test)
            f1weighted=f1_score(y_true, y_pred, average='weighted')
            #print(filename,";",rf_model.score(X_test,y_test),";",f1weighted)
            accuracy[i]=rf_model.score(X_test,y_test)
            #accB[i]=balanced_accuracy_score(y_true, y_pred)
            fscore[i]=f1weighted
        print(filename,";",accuracy.mean(),";",accuracy.std(),";",fscore.mean(),";",fscore.std())     
    

dataImputed ; accuracy.mean() ; accuracy.std() ; fscore.mean() ; fscore.std()
Back_fill05.csv ; 0.8873611111111112 ; 0.022711730972460736 ; 0.8839064220236488 ; 0.017637842232785323
Back_fill10.csv ; 0.8746875 ; 0.04419211410941862 ; 0.8720255521535123 ; 0.03244497259636346
Back_fill25.csv ; 0.8880555555555555 ; 0.02466607622541584 ; 0.8866361313280418 ; 0.01931555898079021
Back_fill40.csv ; 0.8818055555555555 ; 0.03465695254300213 ; 0.8788953248888041 ; 0.025622631479860384
dataCompletTo20maiScaled.csv ; 0.8938541666666667 ; 0.03944054717151212 ; 0.8905117170339352 ; 0.030566855698747383
Forward_fill05.csv ; 0.896701388888889 ; 0.020818426379748612 ; 0.8926370199352489 ; 0.017312950455426166
Forward_fill10.csv ; 0.8887847222222222 ; 0.026187116957038344 ; 0.8843114157814199 ; 0.02085922360900397
Forward_fill25.csv ; 0.884513888888889 ; 0.0234534153781754 ; 0.8830159314437538 ; 0.020345781134314917
Forward_fill40.csv ; 0.8838541666666666 ; 0.031127169869577056 ; 0.8774223810845669 ; 0.

In [142]:
db2

time
1617236497   NaN
1617236799   NaN
1617237102   NaN
1617237405   NaN
1617237707   NaN
              ..
1621609637   NaN
1621609939   NaN
1621610243   NaN
1621610545   NaN
1621610847   NaN
Name: activity, Length: 14397, dtype: float64

In [138]:
y_train

time
1617236497   NaN
1617236799   NaN
1617237102   NaN
1617237405   NaN
1617237707   NaN
              ..
1620739462   NaN
1620739765   NaN
1620740067   NaN
1620740370   NaN
1620740673   NaN
Name: activity, Length: 11517, dtype: float64

In [ ]:
# Kalman filter example demo in Python

# A Python implementation of the example given in pages 11-15 of "An
# Introduction to the Kalman Filter" by Greg Welch and Gary Bishop,
# University of North Carolina at Chapel Hill, Department of Computer
# Science, TR 95-041,
# https://www.cs.unc.edu/~welch/media/pdf/kalman_intro.pdf

# by Andrew D. Straw

import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 8)

# intial parameters
n_iter = data.Temperature.shape[0]
sz = (n_iter,) # size of array
x = -0.37727 # truth value (typo in example at top of p. 13 calls this z)
z = df.Noise # observations (normal about x, sigma=0.1)

Q = 1e-5 # process variance
Q = 0.005

# allocate space for arrays
xhat=np.zeros(sz)      # a posteri estimate of x
P=np.zeros(sz)         # a posteri error estimate
xhatminus=np.zeros(sz) # a priori estimate of x
Pminus=np.zeros(sz)    # a priori error estimate
K=np.zeros(sz)         # gain or blending factor

R = 0.1**2 # estimate of measurement variance, change to see effect

# intial guesses
xhat[0] = 0.0
P[0] = 1.0

for k in range(1,n_iter):
    # time update
    xhatminus[k] = xhat[k-1]
    Pminus[k] = P[k-1]+Q

    # measurement update
    K[k] = Pminus[k]/( Pminus[k]+R )
    xhat[k] = xhatminus[k]+K[k]*(z[k]-xhatminus[k])
    P[k] = (1-K[k])*Pminus[k]

plt.figure()
plt.plot(z,'k+',label='noisy measurements')
xhat[0]= z[0]
plt.plot(xhat,'b-',label='a posteri estimate')
plt.axhline(x,color='g',label='truth value')
plt.legend()
plt.title('Estimate vs. iteration step', fontweight='bold')
plt.xlabel('Iteration')
plt.ylabel('Voltage')

plt.figure()
valid_iter = range(1,n_iter) # Pminus not valid at step 0
plt.plot(valid_iter,Pminus[valid_iter],label='a priori error estimate')
plt.title('Estimated $\it{\mathbf{a \ priori}}$ error vs. iteration step', fontweight='bold')
plt.xlabel('Iteration')
plt.ylabel('$(Voltage)^2$')
plt.setp(plt.gca(),'ylim',[0,.01])
plt.show()


In [131]:
X_train

,Temperature,Humidity,CO2,Noise,Pressure,Temperature_t-1,Humidity_t-1,CO2_t-1,Noise_t-1,Pressure_t-1,...,CO2_t-2,Noise_t-2,Pressure_t-2,Temperature_t-3,Humidity_t-3,CO2_t-3,Noise_t-3,Pressure_t-3,JourSemaine,heureday
time,,,,,,,,,,,,,,,,,,,,,
1617236497,1.261778,1.023008,-0.194203,-0.545073,0.266354,1.297002,1.009896,-0.229277,-0.545073,0.279647,...,-0.267686,-0.545073,0.266354,1.297002,1.009896,-0.020775,-0.545073,0.266354,4,2.350000
1617236799,1.228606,1.009896,-0.210659,-0.545073,0.266354,1.261778,1.023008,-0.194203,-0.545073,0.266354,...,-0.229277,-0.545073,0.279647,1.297002,1.009896,-0.267686,-0.545073,0.266354,4,2.433333
1617237102,1.228606,0.931243,-0.263903,-0.545073,0.266354,1.228606,1.009896,-0.210659,-0.545073,0.266354,...,-0.194203,-0.545073,0.266354,1.297002,1.009896,-0.229277,-0.545073,0.279647,4,2.516667
1617237405,1.228606,0.860782,-0.339015,-0.545073,0.266354,1.228606,0.931243,-0.263903,-0.545073,0.266354,...,-0.210659,-0.545073,0.266354,1.261778,1.023008,-0.194203,-0.545073,0.266354,4,2.600000
1617237707,1.228606,0.860782,-0.388397,-0.545073,0.269137,1.228606,0.860782,-0.339015,-0.545073,0.266354,...,-0.263903,-0.545073,0.266354,1.228606,1.009896,-0.210659,-0.545073,0.266354,4,2.683333
1617238010,1.243490,0.862736,-0.207330,-0.545073,0.266354,1.228606,0.860782,-0.388397,-0.545073,0.269137,...,-0.339015,-0.545073,0.266354,1.228606,0.931243,-0.263903,-0.545073,0.266354,4,2.766667
1617238313,1.228606,0.860782,-0.227540,-0.545073,0.253061,1.243490,0.862736,-0.207330,-0.545073,0.266354,...,-0.388397,-0.545073,0.269137,1.228606,0.860782,-0.339015,-0.545073,0.266354,4,2.850000
1617238615,1.160209,0.860464,-0.339015,-0.545073,0.253061,1.228606,0.860782,-0.227540,-0.545073,0.253061,...,-0.207330,-0.545073,0.266354,1.228606,0.860782,-0.388397,-0.545073,0.269137,4,2.933333
1617238918,1.160209,0.860782,-0.353958,-0.545073,0.266354,1.160209,0.860464,-0.339015,-0.545073,0.253061,...,-0.227540,-0.545073,0.253061,1.243490,0.862736,-0.207330,-0.545073,0.266354,4,3.016667


In [118]:
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        df = pd.read_csv(f, sep=",", encoding = "ISO-8859-1")        
        #df= pd.DataFrame(datascaler.inverse_transform(df.values[:,1:]),columns=df.columns[1:])        
        df, lignesAsupprimer=lagged(df,lagg)
        print(df.head(2))

            Temperature  Humidity       CO2     Noise  Pressure  \
time                                                              
1617236497     1.297002  1.009896 -0.207330 -0.545073  0.266354   
1617236799     1.228606  1.009896 -0.146974 -0.545073  0.266354   

            Temperature_t-1  Humidity_t-1   CO2_t-1  Noise_t-1  Pressure_t-1  \
time                                                                           
1617236497         1.297002      1.009896 -0.229277  -0.545073      0.279647   
1617236799         1.297002      1.009896 -0.207330  -0.545073      0.266354   

            Temperature_t-2  Humidity_t-2   CO2_t-2  Noise_t-2  Pressure_t-2  \
time                                                                           
1617236497         1.297002      1.009896 -0.267686  -0.545073      0.266354   
1617236799         1.297002      1.009896 -0.229277  -0.545073      0.279647   

            Temperature_t-3  Humidity_t-3   CO2_t-3  Noise_t-3  Pressure_t-3  
time      

In [117]:
df

,Temperature,Humidity,CO2,Noise,Pressure,Temperature_t-1,Humidity_t-1,CO2_t-1,Noise_t-1,Pressure_t-1,Temperature_t-2,Humidity_t-2,CO2_t-2,Noise_t-2,Pressure_t-2,Temperature_t-3,Humidity_t-3,CO2_t-3,Noise_t-3,Pressure_t-3
time,,,,,,,,,,,,,,,,,,,,
1617236497,1.261778,1.023008,-0.194203,-0.545073,0.266354,1.297002,1.009896,-0.229277,-0.545073,0.279647,1.297002,1.009896,-0.267686,-0.545073,0.266354,1.297002,1.009896,-0.020775,-0.545073,0.266354
1617236799,1.228606,1.009896,-0.210659,-0.545073,0.266354,1.261778,1.023008,-0.194203,-0.545073,0.266354,1.297002,1.009896,-0.229277,-0.545073,0.279647,1.297002,1.009896,-0.267686,-0.545073,0.266354
1617237102,1.228606,0.931243,-0.263903,-0.545073,0.266354,1.228606,1.009896,-0.210659,-0.545073,0.266354,1.261778,1.023008,-0.194203,-0.545073,0.266354,1.297002,1.009896,-0.229277,-0.545073,0.279647
1617237405,1.228606,0.860782,-0.339015,-0.545073,0.266354,1.228606,0.931243,-0.263903,-0.545073,0.266354,1.228606,1.009896,-0.210659,-0.545073,0.266354,1.261778,1.023008,-0.194203,-0.545073,0.266354
1617237707,1.228606,0.860782,-0.388397,-0.545073,0.269137,1.228606,0.860782,-0.339015,-0.545073,0.266354,1.228606,0.931243,-0.263903,-0.545073,0.266354,1.228606,1.009896,-0.210659,-0.545073,0.266354
1617238010,1.243490,0.862736,-0.207330,-0.545073,0.266354,1.228606,0.860782,-0.388397,-0.545073,0.269137,1.228606,0.860782,-0.339015,-0.545073,0.266354,1.228606,0.931243,-0.263903,-0.545073,0.266354
1617238313,1.228606,0.860782,-0.227540,-0.545073,0.253061,1.243490,0.862736,-0.207330,-0.545073,0.266354,1.228606,0.860782,-0.388397,-0.545073,0.269137,1.228606,0.860782,-0.339015,-0.545073,0.266354
1617238615,1.160209,0.860464,-0.339015,-0.545073,0.253061,1.228606,0.860782,-0.227540,-0.545073,0.253061,1.243490,0.862736,-0.207330,-0.545073,0.266354,1.228606,0.860782,-0.388397,-0.545073,0.269137
1617238918,1.160209,0.860782,-0.353958,-0.545073,0.266354,1.160209,0.860464,-0.339015,-0.545073,0.253061,1.228606,0.860782,-0.227540,-0.545073,0.253061,1.243490,0.862736,-0.207330,-0.545073,0.266354


In [130]:
lignesAsupprimer

[0, 1, 2]